In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print(device)

with open('data/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))

d_input = len(chars) + 1 # let 0 be padding
print(d_input)

stoi = lambda c: chars.index(c) + 1
itos = lambda n: "" if n == 0 else chars[n-1]
encode = lambda s: torch.tensor([stoi(c) for c in s], dtype=torch.long)
decode = lambda m: ''.join([itos(i) for i in m])

code = encode('abc')
print(code)
print(decode(code))

data = encode(text)
n_split = int(0.9 * len(data))
train_data = data[:n_split]
test_data = data[n_split:]

def get_batch(mode, seq_len, batch_size=1):
    source = train_data if mode == 'train' else test_data
    starts = torch.randint(len(source) - seq_len - 1, (batch_size, ))
    x = torch.stack([source[s:s+seq_len] for s in starts])
    y = torch.stack([source[s+1:s+1+seq_len] for s in starts])
    x, y = x.to(device), y.to(device)
    return x, y

eval_iters = 10 #200
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for mode in ['train', 'eval']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(mode, max_seq_len)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

cuda
66
tensor([40, 41, 42])
abc


In [2]:
import gpt

max_seq_len = 64

model = gpt.GPT(d_model=32, d_input=d_input, max_seq_len=max_seq_len, N=6)
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)

In [3]:
max_iters = 30
for iter in range(1, max_iters+1):
    
    x, y = get_batch('train', max_seq_len-3, 20)

    z, loss = model(x, y)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if iter % (max_iters // 10) == 0 or iter == 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['eval']:.4f}")

pos encoding device cuda:0
layer norm device cuda:0
mask device cuda:0
layer norm device cuda:0
layer norm device cuda:0
mask device cuda:0
layer norm device cuda:0
layer norm device cuda:0
mask device cuda:0
layer norm device cuda:0
layer norm device cuda:0
mask device cuda:0
layer norm device cuda:0
layer norm device cuda:0
mask device cuda:0
layer norm device cuda:0
layer norm device cuda:0
mask device cuda:0
layer norm device cuda:0
layer norm device cuda:0


TypeError: forward() missing 1 required positional argument: 'device'

In [ ]:
context = encode("Hi").unsqueeze(0).to(device) # add batch to 1

new_seq_len=300
for _ in range(new_seq_len):
    z = context
    if z.size()[1] > max_seq_len:
        z = z[:,-max_seq_len:]
    z, _ = model(z)
    z = z[:, -1, :]
    probs = F.softmax(z, dim=-1)
    char_vecs = torch.multinomial(probs, num_samples=1)
    context = torch.cat([context, char_vecs], dim=-1)
    
result = context[0]
print(decode(result))

StopIteration: 